### Lecture 4: Homework

Today we gonna learn how to choose between ML models, based on data type. Your task would be to predict **the edibility of a mushroom** based on sample descriptions (binary classification problem)

The **tricky part here is that 95% of the features are of categorical type.**
<br>That's the one where we would **(usually)  prefer tree-based algorithms over linear methods**

Although this dataset was originally contributed to the UCI Machine Learning repository nearly 30 years ago, mushroom hunting (otherwise known as "shrooming") is enjoying new peaks in popularity. Learn which features spell certain death and which are most palatable in this dataset of mushroom characteristics. And how certain can your model be?

This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom drawn from The Audubon Society Field Guide to North American Mushrooms (1981). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be'' for Poisonous Oak and Ivy.

More information can be found [here](https://www.kaggle.com/uciml/mushroom-classification/data)

Please find below correspondent [google form](https://docs.google.com/forms/d/e/1FAIpQLScmKfUApMlcD81u9UZxM7xG3vJiEJHrPrG-3b0i_jyPEDijgQ/viewform) to submit your answers

In [1]:
# library import
import pandas as pd
import numpy as np
from os.path import join as pjoin
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

# preprocessing / validation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
)
# ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# metrics
from sklearn.metrics import classification_report, f1_score

In [2]:
# read data
df_train = pd.read_csv('4-mushrooms-train.csv')
df_test = pd.read_csv('4-mushrooms-test.csv')
print(df_train.shape, df_test.shape)

(6499, 23) (1625, 22)


In [3]:
# let's see what data looks like
df_train.head()

,target,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,convex,scaly,brown,bruises,pungent,free,close,narrow,white,enlarging,equal,smooth,smooth,white,white,partial,white,one,pendant,brown,scattered,urban
1,1,flat,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,brown,several,woods
2,0,flat,smooth,brown,no,none,attached,close,broad,orange,enlarging,missing,smooth,smooth,orange,orange,partial,orange,one,pendant,brown,several,leaves
3,1,convex,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,black,solitary,woods
4,0,knobbed,smooth,brown,no,foul,free,close,narrow,buff,tapering,missing,silky,smooth,pink,pink,partial,white,one,evanescent,white,several,paths


In [4]:
# for convenient calculations, let us merge train with test
df = pd.concat([df_train, df_test], axis=0)
# add column for filtering train/test
df['is_train'] = True
df.loc[df.target.isnull(), 'is_train'] = False
# check shapes
print(df.shape)
# check labels
df.is_train.value_counts()

(8124, 24)


True     6499
False    1625
Name: is_train, dtype: int64

### Task 1. Which feature has the highest amount of unique values? (joint dataset)


In [5]:
features = df.drop(['target', 'is_train'], 1).columns.values

In [6]:
unq_vals = {}
for feature in features:
    unq_vals[feature] = len(df[feature].value_counts())

In [7]:
# your code goes here
# ---------------------------------------------------------------
most_diversive = sorted(unq_vals, key=unq_vals.get, reverse=True)[0]
# ---------------------------------------------------------------

print(most_diversive)

gill_color


### Task 2
**As a preparation, one would spend up to 15-30 minutes on exploratory data analysis (EDA)** - make sure you understand how features are distributed in train/test, what they look like, are they ordinal/binary/categorical before moving further
<br>While doing it, please answer the questions

In [8]:
for col in df.drop(['target', 'is_train'], 1).columns:
    print(col, 'has {} value(s).'.format(len(df[col].value_counts())))

bruises has 2 value(s).
cap_color has 10 value(s).
cap_shape has 6 value(s).
cap_surface has 4 value(s).
gill_attachment has 2 value(s).
gill_color has 12 value(s).
gill_size has 2 value(s).
gill_spacing has 2 value(s).
habitat has 7 value(s).
odor has 9 value(s).
population has 6 value(s).
ring_number has 3 value(s).
ring_type has 5 value(s).
spore_print_color has 9 value(s).
stalk_color_above_ring has 9 value(s).
stalk_color_below_ring has 9 value(s).
stalk_root has 5 value(s).
stalk_shape has 2 value(s).
stalk_surface_above_ring has 4 value(s).
stalk_surface_below_ring has 4 value(s).
veil_color has 4 value(s).
veil_type has 1 value(s).


'veil_type' has only one value, that's why it won't decrease impurity of samples in any leaf at all.

#### 2.1 Are there any features, obviously redundant to train on? If yes - what are they and why it's better to remove them?

In [9]:
print(df.shape)
# your code/hardcoded list goes here
# ---------------------------------------------------------------
redundant_columns = ['veil_type']
# ---------------------------------------------------------------
# lets drop these columns from joint dataset
df_new = df.drop(redundant_columns, axis=1)
print(df_new.shape)

(8124, 24)
(8124, 23)


####  2.2 How many features (excluding target variable and train/test indexing columns) are:
- categorical (more than 2 unique values, no explicit ordering)
- ordinal (more than 2 unique values, explicit ordering)
- binary (2 unique values, doesn't matter whether it has ordering or is "yes/no" styled) 

In [10]:
# your code goes here
# ---------------------------------------------------------------
ordinal_cols = ['ring_number']
binary_cols = sorted([col for col in df_new.drop(['target', 'is_train'], 1).columns if len(df_new[col].value_counts()) == 2])
categorical_cols = sorted(df_new.drop(ordinal_cols+binary_cols+['is_train', 'target'], 1).columns.values.tolist())
# ---------------------------------------------------------------
print('categorical: {} \nordinal: {} \nbinary: {}'.format(
    len(categorical_cols), len(ordinal_cols), len(binary_cols)))

categorical: 15 
ordinal: 1 
binary: 5


In [11]:
# To be used in training, data must be properly encoded
from collections import defaultdict

# function to encode categorical data
def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))


# label encode data (categorical + binary)
__encode_categorical(df_list=[df_new], cat_cols=categorical_cols+binary_cols)
# make sure you encode the only ordinal column in correct order
df_new[ordinal_cols[0]] = df_new[ordinal_cols[0]].map({'none': 0, 'one': 1, 'two': 2})

# define useful feature columns to be used for training
# (union of all columns discussed above)
columns_to_use = binary_cols + categorical_cols + ordinal_cols

And let's make chi-squared test now.

In [12]:
from sklearn.feature_selection import chi2
chi_test = chi2(df_new[df_new.is_train == True][columns_to_use], df_new[df_new.is_train==True]['target'])
chi_result = pd.Series(index=pd.Index(columns_to_use), data=chi_test[1])

In [13]:
#top-3 redundant features 
#their p-values are much bigger than those of other features
top3 = chi_result.sort_values(ascending=False).head(3).index.values.tolist() 

In [14]:
top3

['gill_attachment', 'veil_color', 'habitat']

In [15]:
#dropping 3 features
df_new = df_new.drop(top3, 1)

In [16]:
columns_to_use = [col for col in columns_to_use if col not in top3] #update list of features

### Task 3. Prepare cross-validation strategy and perform comparison of 2 baseline models (linear vs tree-based)

### =====================================================
#### Briefly about Validation / Cross-Validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but **would fail to predict anything useful on yet-unseen data. This situation is called overfitting**. 
<br>To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set ```X_test, y_test```. 
<br>Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally.

When evaluating different settings (“hyperparameters”) for estimators, **there is still a risk of overfitting on the test set** because the parameters can be tweaked until the estimator performs optimally. 
<br>This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. 
<br>To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, **we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.**

A solution to this problem is a procedure called **cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV**. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

- A model is trained using k-1 of the folds as training data;
- the resulting model is validated on the remaining part of the data 
<br>(i.e., it is used as a test set to compute a performance measure such as accuracy).
        
<img src="https://hsto.org/files/b1d/706/e6c/b1d706e6c9df49c297b6152878a2d03f.png"/ style="width:75%">

The performance measure reported by k-fold cross-validation **is then the average of the values computed in the loop**. 
<br>This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.


Some classification problems can **exhibit a large imbalance in the distribution of the target classes: for instance there could be several times more negative samples than positive samples**. 
<br>In such cases it is recommended to use **stratified sampling** as implemented in sklearn's StratifiedKFold and StratifiedShuffleSplit to ensure that relative class frequencies is approximately preserved in each train and validation fold.

More details about different cross-validation strategies, implemented in sklearn, can be found [here](http://scikit-learn.org/stable/modules/cross_validation.html)
### =====================================================

Prepare KFold with 5 splits, stratified by target variable, shuffled, with fixed random_state = 42
<br>**Don't forget to filter by column 'is_train'!**
<br>Fit models on subset of features: [columns_to_use]

In [17]:
from os import cpu_count

n_jobs = max(cpu_count()-1, 1)
# cross-validation iterator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# xtrain, ytrain, DataFrame-like
xtrain = df_new[df_new.is_train][columns_to_use]
ytrain = df_new[df_new.is_train]['target']
# ---------------------------------------------------------------

# create Decision Tree with default params, max_depth=3, random_state=42
dt = DecisionTreeClassifier(
    max_depth=3,
    random_state=42
)

# estimate its f1-score with cross-validation (cross_val_score)
# your code goes here
# ---------------------------------------------------------------
scores_dt = cross_val_score(
    estimator=dt,
    X=xtrain,
    y=ytrain, 
    scoring='f1', 
    cv=kf, # cross-validation strategy
    n_jobs=n_jobs
).mean()
print('DT scoring: {:.2f}'.format(scores_dt))
# ---------------------------------------------------------------


# create Logistic Regression with default params, random_state=42
lr = LogisticRegression(
    random_state=42
)

# estimate its f1-score with cross-validation
# your code goes here
# ---------------------------------------------------------------
scores_lr = cross_val_score(
    estimator=lr,
    X=xtrain, # ...
    y=ytrain, # ...
    scoring='f1',
    cv=kf,
    n_jobs=n_jobs
).mean()
print('LR scoring: {:.2f}'.format(scores_lr))

DT scoring: 0.90
LR scoring: 0.89


Why is a score of Linear Regression lower than correspondent one of DT?
1. Is everything OK with the data format for linear models? (revision of 2 previous lectures). 
1. If not, what else you should do to use the data appropriately for linear models?
1. Why didn't point 1. affect Decision Tree performance?

1. No, it should be one-hot encoded.
2. Make one-hot encoding for all features.
3. Because decision tree classification based on rules which split data the best way, not on probabilities of object to have one or another feature.

In [18]:
hot_enc = OneHotEncoder()
hot_enc.fit(df_new[df_new.is_train][columns_to_use]) 
train_hot = hot_enc.transform(df_new[df_new.is_train][columns_to_use]) #one-hot encoding of train data

In [19]:
%%time
new_lr = LogisticRegression(
    random_state=42
)
scores_new_lr = cross_val_score(
    estimator=new_lr,
    X=train_hot,
    y=ytrain,
    scoring='f1',
    cv=kf
).mean()
print('LR with one-hot encoded data: {:.2f}'.format(scores_new_lr))

LR with one-hot encoded data: 0.93
Wall time: 193 ms


Simple logistic regression fitted by one-hot encoded train data performs better on CV than tree.

### Task 4. Now it's time to do some hyperparam tuning
Perform suitable hyperparam tuning using created above cross-validation strategy
<br>Main parameters to perform grid-search for:
- max_depth (1,2,...None)
- min_samples_leaf (1,2,...)
- criterion (gini, entropy)
- weight (none, balanced)
- max_features (sqrt(features), 50%, 75%, all of them, ...)
- other params available, see documentation

So - use your fantasy for filling-in abovementioned lists

You should receive **a gain of 0.01 in f1-score or higher**
<br>(current benchmark = +0.0268 gain)

In [20]:
%%time
# your code goes here
# ---------------------------------------------------------------
# create base model (DT, random state = 42)
estimator = DecisionTreeClassifier(
    random_state=42
)

# create parameter grid
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
params = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': list(range(1, 11)) + [None],
    'min_samples_split': list(range(2, 10)),
    'min_samples_leaf': list(range(1, 10)),
    'max_features': ['sqrt', 'log2', 0.5, 0.75, 0.8, 0.85, 0.9, None],
    'class_weight': ['balanced', None]
}

# create grid search object
gs = GridSearchCV(
    estimator=estimator,  # base model
    param_grid=params,  # params grid to search within
    cv=kf,  # cross-validation strategy
    error_score=1,  # warnings only
    scoring='f1',  # f1-score
    # thread count, the higher count - the faster
    n_jobs=n_jobs,
    verbose=2,  # messages about performed actions
)

# perform grid search on TRAIN dataset ('is_train' filtering)
gs.fit(
    X=xtrain,
    y=ytrain, # ..
)
# -------------------------------------------------------------
# extract best score on cross-validation
best_score = gs.best_score_
# extract the estimator (DT) with best params on cross-validation
best_dt = gs.best_estimator_
# check gain in f1-score
print('f1-score best: {:.4f}, +{:.4f} better than baseline'.format(
    best_score, (best_score - scores_dt))
)

Fitting 5 folds for each of 50688 candidates, totalling 253440 fits


[Parallel(n_jobs=3)]: Done 131 tasks      | elapsed:    2.6s
[Parallel(n_jobs=3)]: Done 2067 tasks      | elapsed:   11.3s
[Parallel(n_jobs=3)]: Done 5315 tasks      | elapsed:   25.9s
[Parallel(n_jobs=3)]: Done 9843 tasks      | elapsed:   45.1s
[Parallel(n_jobs=3)]: Done 15683 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done 22803 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done 31235 tasks      | elapsed:  2.7min
[Parallel(n_jobs=3)]: Done 40947 tasks      | elapsed:  3.7min
[Parallel(n_jobs=3)]: Done 51971 tasks      | elapsed:  4.8min
[Parallel(n_jobs=3)]: Done 64275 tasks      | elapsed:  6.2min
[Parallel(n_jobs=3)]: Done 77891 tasks      | elapsed:  7.3min
[Parallel(n_jobs=3)]: Done 92787 tasks      | elapsed:  8.6min
[Parallel(n_jobs=3)]: Done 108995 tasks      | elapsed: 10.2min
[Parallel(n_jobs=3)]: Done 126483 tasks      | elapsed: 12.3min
[Parallel(n_jobs=3)]: Done 145283 tasks      | elapsed: 13.6min
[Parallel(n_jobs=3)]: Done 165363 tasks      | elapsed: 1

f1-score best: 0.9283, +0.0291 better than baseline
Wall time: 22min 50s


In [23]:
# take a look on the best model, compare with the baseline
best_dt

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=8, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=8, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [24]:
# check performance on holdout dataset, unseen before (filter 'is_train' == False)

# your code goes here
# ---------------------------------------------------------------
# appropriate df_test data subset from 'df' dataframe
xtest = df_new[df_new.is_train == False][columns_to_use] # ...
# fit baseline model 'dt' on xtrain, ytrain (because it's not fitted yet)
dt.fit(xtrain, ytrain)
# ---------------------------------------------------------------

# baseline model
y_true = pd.read_csv('4-mushrooms-y_test.csv')
y_pred_baseline = dt.predict(xtest)

print('Base on train:   {:.4f}\nBase on holdout: {:.4f}\ndiff: {:.4f}'.format(
    scores_dt, 
    f1_score(y_true, y_pred_baseline),
    scores_dt - f1_score(y_true, y_pred_baseline)
))

# best model
y_pred_best = best_dt.predict(xtest)

print('\nBest on train:   {:.4f}\nBest on holdout: {:.4f}\ndiff: {:.4f}\n'.format(
    best_score, 
    f1_score(y_true, y_pred_best),
    best_score - f1_score(y_true, y_pred_best)
))

print('Detailed report (baseline):\n{}\nDetailed report (best tree):\n{}'.format(
    classification_report(y_true, y_pred_baseline),
    classification_report(y_true, y_pred_best)
))

Base on train:   0.8992
Base on holdout: 0.8872
diff: 0.0119

Best on train:   0.9283
Best on holdout: 0.9205
diff: 0.0078

Detailed report (baseline):
             precision    recall  f1-score   support

          0       0.86      0.92      0.89       791
          1       0.92      0.86      0.89       834

avg / total       0.89      0.89      0.89      1625

Detailed report (best tree):
             precision    recall  f1-score   support

          0       0.91      0.92      0.92       791
          1       0.92      0.92      0.92       834

avg / total       0.92      0.92      0.92      1625



Now you can see that 
<br>**absolute values of f1-score is higher and distance between train|holdout is lower** <br>for **best model** in comparison to **baseline**

**Bonus question**:

Consider two possibilities:
- (a) you have trained **one best** (on cross-validation) Decision Tree
- (b) you randomly choose 25 subsets of 70% of training data, fits "overfitted" (max_depth=None) Decision Trees on it - each of them performs slightly worse than Tree in (a), and then average predicted results over all 25 models (overfitted trees)

**Which one of them would most likely give the best results on hold-out dataset? What makes you think that way?**

As seen below, forest performs on holdout dataset worse than best tree, but better than baseline tree.
Each of 25 overfitted trees trained on one small subset of data, that's why each tree perfectly predicts class of mushrooms with certain values of features.

In [52]:
from sklearn.utils import shuffle
df_shuffle = shuffle(df_new[df_new.is_train], random_state=42) #shuffle training set

In [53]:
from math import ceil
train_set_size = ceil(df_shuffle.shape[0]*.7) #taking 70% of train data
dftrain_ = df_shuffle.iloc[0:train_set_size, :] #new train

In [54]:
xtrain_ = dftrain_[columns_to_use]
ytrain_ = dftrain_['target']

In [55]:
batch_size = dftrain_.shape[0]/25 #size of data for each tree 

In [56]:
forest = {} #dict of fitted trees
for idx in range(0, 25):
    forest[idx+1] = DecisionTreeClassifier(
        max_depth=None,
        random_state=42
)
    xbatch = xtrain_.iloc[int(idx*batch_size):int((idx+1)*batch_size), :]
    ybatch = ytrain_.iloc[int(idx*batch_size):int((idx+1)*batch_size)]
    forest[idx+1].fit(xbatch, ybatch)

In [57]:
pred_hold = {} #predictions of each tree
for model in forest:
    pred_hold[model] = forest[model].predict(xtest)
    score = f1_score(y_true, pred_hold[model])
    print('Model {} has score {}'.format(model, score))

Model 1 has score 0.8120024494794855
Model 2 has score 0.8139390168014935
Model 3 has score 0.8281157708210278
Model 4 has score 0.813968253968254
Model 5 has score 0.8048632218844984
Model 6 has score 0.7842445620223398
Model 7 has score 0.868208092485549
Model 8 has score 0.8506024096385543
Model 9 has score 0.8292978208232445
Model 10 has score 0.8253557567917206
Model 11 has score 0.8052930056710775
Model 12 has score 0.8331288343558282
Model 13 has score 0.8365155131264916
Model 14 has score 0.7970660146699267
Model 15 has score 0.8219696969696971
Model 16 has score 0.837037037037037
Model 17 has score 0.7787286063569683
Model 18 has score 0.8327272727272728
Model 19 has score 0.8614045991298944
Model 20 has score 0.8246674727932285
Model 21 has score 0.849009900990099
Model 22 has score 0.8264264264264265
Model 23 has score 0.8105263157894738
Model 24 has score 0.8379544054220579
Model 25 has score 0.8233815210559396


Each model has much smaller f1-score than baseline and best model.

In [58]:
pred_hold = pd.DataFrame.from_dict(pred_hold)

In [59]:
f1_score(y_true, pred_hold.mean(1).apply(round)) #f1-score for averaged result of all calssifiers

0.9188214070956103

In [61]:
print('Detailed report (forest):\n{}'.format(
    classification_report(y_true, pred_hold.mean(1).apply(round))
))
print('Forest is better than baseline on {:.2f} points (for holdout data)'.format(
    f1_score(y_true, pred_hold.mean(1).apply(round))-f1_score(y_true, y_pred_baseline)
))

Detailed report (forest):
             precision    recall  f1-score   support

          0       0.91      0.92      0.91       791
          1       0.92      0.92      0.92       834

avg / total       0.92      0.92      0.92      1625

Forest is better than baseline on 0.03 points (for holdout data)
